In [1]:
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import json
import langid


In [2]:
# imports for analysis
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim
from IPython.core.display import HTML

# NOTES !
# You might need to run these for spaCy to work!
# !pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz
# ensure you have the latest version of PANDAS!


In [3]:
# 1. Data Collection --------------------------------------------------------------------------------

# Loading json
with open('Science_Technology_News.json', encoding='utf-8') as f:
    data = json.load(f)

# Main dataframe
df = pd.json_normalize(data)

# Printing Main dataframe structure
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5437 entries, 0 to 5436
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             5437 non-null   object
 1   link              5437 non-null   object
 2   keywords          2502 non-null   object
 3   creator           2610 non-null   object
 4   video_url         0 non-null      object
 5   description       4936 non-null   object
 6   content           1627 non-null   object
 7   pubDate           5437 non-null   object
 8   full_description  2670 non-null   object
 9   image_url         2169 non-null   object
 10  source_id         5437 non-null   object
dtypes: object(11)
memory usage: 467.4+ KB
None


In [4]:
# 2. Information Extraction -----------------------------------------------------------------

# Extracting relevant fields
df = df[['title', 'pubDate', 'creator', 'content']].copy()

# Filtering out non-English articles ------------------------------------------
def englishfilter(text):
    if pd.isna(text):
        return "other"
    else:
        return langid.classify(text)[0]
df['language'] = df['content'].apply(englishfilter)
df = df[df['language'] == 'en']
df.drop('language', axis=1, inplace=True)
#-----------------------------------------------------------------------------

# Preprocessing
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    if text:
        tokens = tokenizer.tokenize(text.lower())
        filtered_tokens = [word for word in tokens if word not in stop_words]
        stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
        return " ".join(stemmed_tokens)
    return ""

def preprocess_lemmatize(text):
    if text:
        tokens = tokenizer.tokenize(text.lower())
        filtered_tokens = [word for word in tokens if word not in stop_words]
        lemmed_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
        return " ".join(lemmed_tokens)
    return ""

# Applying preprocessing to content field
df['processed_content_stemmed'] = df['content'].apply(preprocess)

# Applying preprocessing to content field
df['processed_content_lemmed'] = df['content'].apply(preprocess_lemmatize)


In [5]:
# Printing Main dataframe structure
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 565 entries, 19 to 5423
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   title                      565 non-null    object
 1   pubDate                    565 non-null    object
 2   creator                    486 non-null    object
 3   content                    565 non-null    object
 4   processed_content_stemmed  565 non-null    object
 5   processed_content_lemmed   565 non-null    object
dtypes: object(6)
memory usage: 30.9+ KB
None


In [6]:
# Printing preprocessed content
print(df['processed_content_stemmed'].head())
print(df['processed_content_lemmed'].head())

19    african automot compani autochek secur 13 1 mi...
25    right schedul spacex roll new crew dragon spac...
26    jeff bezo offici announc plan space ventur blu...
68    hasbro offici partner worldwid asset exchang w...
80    new seri cyclingnew delv back catalogu profess...
Name: processed_content_stemmed, dtype: object
19    african automotive company autochek secured 13...
25    right schedule spacex rolled new crew dragon s...
26    jeff bezos officially announced plan space ven...
68    hasbro officially partnered worldwide asset ex...
80    new series cyclingnews delving back catalogue ...
Name: processed_content_lemmed, dtype: object


In [7]:
## Topic Modeling ----------------------------------------------------------------------------------------------------------

# Restructuring of the data is nessecary 
# will be using the lammetized data (better for topic modeling)
# will be dropping all Nan values for data uniformity
# will be combining titles, dates, etc for training

unframed_data = df.drop(["content", "processed_content_stemmed"], axis=1).dropna().values.tolist()

# set up text_corpus
text_corpus = []

for article in unframed_data:
    new_article = []
    new_article.append(article[0].split(" "))
    new_article = new_article + article[1].split(" ")
    new_article = new_article + article[2][0].split(" ")
    new_article = new_article[0] + article[3].split(" ")
    text_corpus.append(new_article)

# suppressed, for debugging purposes only
# print(text_corpus[0])

In [8]:
# Perform mapping of ID to words
id2word = corpora.Dictionary(text_corpus)

corpus = []
for article in text_corpus:
    new = id2word.doc2bow(article)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 2), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 2), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 2)]
$13.1M


In [9]:
# Actual Topic Model using LDA

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [10]:
## Topic Visualization
pyLDAvis.enable_notebook(local=True)
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

C:\ProgramData\Anaconda3\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
C:\Users\weden\AppData\Roaming\Python\Python37\site-packages\pyLDAvis\_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
20     0.160193  0.431558       1        1  23.115657
29     0.422101 -0.146894       2        1  21.231017
21     0.420911  0.106469       3        1  13.472905
22     0.338980  0.275522       4        1   9.531209
13     0.362212 -0.060495       5        1   5.768713
17     0.298128 -0.279807       6        1   5.704441
27     0.162543  0.298004       7        1   3.662240
19     0.287429  0.077719       8        1   3.457133
26    -0.079624 -0.388601       9        1   2.134259
9     -0.210198  0.297904      10        1   2.085542
23    -0.083953  0.306520      11        1   1.799015
15     0.072562 -0.307572      12        1   1.566517
25    -0.296233 -0.176341      13        1   1.017542
4     -0.016237  0.170111      14        1   0.897422
0     -0.279983  0.106474      15        1   0.782790
12     0.040463 -0.044918      16        1   0.474378
16    -0.053001 -0.171048      17        1   0.472228
11    -0.247064 -0.025947      18        1   0.418466
28    -0.148253 -0.155125      19        1   0.371065
24    -0.116304  0.061808      20        1   0.359569
10    -0.029244 -0.001245      21        1   0.326391
7     -0.182098 -0.058828      22        1   0.325340
18    -0.180367  0.041190      23        1   0.324294
14    -0.066897 -0.091167      24        1   0.199458
3     -0.071192 -0.053623      25        1   0.147926
8     -0.118484 -0.078193      26        1   0.136058
2     -0.112954 -0.030852      27        1   0.112968
6     -0.090206 -0.028080      28        1   0.043422
5     -0.093119 -0.039796      29        1   0.040132
1     -0.090110 -0.034745      30        1   0.021901, topic_info=          Term        Freq       Total Category  logprob  loglift
1849     apple  346.000000  346.000000  Default  30.0000  30.0000
3707  facebook  469.000000  469.000000  Default  29.0000  29.0000
375       user  399.000000  399.000000  Default  28.0000  28.0000
1160       pro  312.000000  312.000000  Default  27.0000  27.0000
152      first  496.000000  496.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
164        get    0.001602  385.547359  Topic30  -9.7746  -3.9650
152      first    0.001602  496.164773  Topic30  -9.7746  -4.2172
2941    recent    0.001602   81.505253  Topic30  -9.7746  -2.4110
151       firm    0.001602   45.746538  Topic30  -9.7746  -1.8334
4520  designed    0.001602   54.821515  Topic30  -9.7746  -2.0144

[1523 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
1602      1  0.994495     0
2672      1  0.359155    00
2672      8  0.553698    00
2672     13  0.074824    00
1         1  0.013919   000
...     ...       ...   ...
3124      3  0.148516     z
3124     14  0.816838     z
618       4  0.335626  zero
618      22  0.587346  zero
5821     14  0.901319     –

[2360 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[21, 30, 22, 23, 14, 18, 28, 20, 27, 10, 24, 16, 26, 5, 1, 13, 17, 12, 29, 25, 11, 8, 19, 15, 4, 9, 3, 7, 6, 2])